In [ ]:
import os

In [ ]:
# switch to the path where the dataset is stored
%cd /content/drive/MyDrive/dataset_images/

/content/drive/MyDrive/dataset_images


Don't forget to modify my_dataset.py when you want to change the dataset


In [ ]:
# copy the zip file (dataset) to the /content directory

#!cp SSMFVD.zip /content
!cp kaggle.zip /content
#!cp PWMFD.zip /content

In [ ]:
# # switch to content directory
%cd /content
# unzip
#!unzip SSMFVD.zip -d '/content/dataset'
!unzip kaggle.zip -d '/content/dataset'
#!unzip PWMFD.zip -d '/content/dataset'

In [ ]:
# switch to the path where the dataset txt files are stored

#%cd /content/drive/MyDrive/TrainingStage2/data/own_video_dataset_txt/

%cd /content/drive/MyDrive/TrainingStage2/data/kaggle_dataset_txt

#%cd /content/drive/MyDrive/TrainingStage2/data/PWMFD_dataset_txt

/content/drive/MyDrive/faster_rcnn/data/kaggle_dataset_txt


In [ ]:
# copy the txt files to /content/dataset/
!cp train.txt /content/dataset/ #
!cp valid.txt /content/dataset/ #
!cp test.txt /content/dataset/ #

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Aug 11 01:32:43 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    30W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import sys
print(sys.path)
sys.path.append('/content/drive/MyDrive/TrainingStage2/')
# print(sys.path)

['', '/content', '/env/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.7/dist-packages/IPython/extensions', '/root/.ipython']


In [ ]:
import os
import datetime

import torch
import torchvision

import transforms
from network_files import FasterRCNN, FastRCNNPredictor
from backbone import resnet50_fpn_backbone
from my_dataset import MyDataSet
from train_utils import GroupedBatchSampler, create_aspect_ratio_groups
from train_utils import train_eval_utils as utils

In [ ]:

def create_model(num_classes, load_pretrain_weights=True): # True
    # trainable_layers includes ['layer4', 'layer3', 'layer2', 'layer1', 'conv1']
    # resnet50 imagenet weights url: https://download.pytorch.org/models/resnet50-0676ba61.pth
    backbone = resnet50_fpn_backbone(pretrain_path="/content/drive/MyDrive/TrainingStage2/backbone/resnet50.pth",
                                     norm_layer= torch.nn.BatchNorm2d, # torchvision.ops.FrozenBatchNorm2d, torch.nn.BatchNorm2d
                                     trainable_layers=4)
    # 90+1 or 3+1, set it as 4 when you load your own model，otherwise 91
    model = FasterRCNN(backbone=backbone, num_classes=91)

    if load_pretrain_weights:
        # load pre-trained weights
        # https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
        weights_dict = torch.load("/content/drive/MyDrive/TrainingStage2/backbone/fasterrcnn_resnet50_fpn_coco.pth", map_location='cpu')
        #weights_dict = weights_dict["model"]
        missing_keys, unexpected_keys = model.load_state_dict(weights_dict, strict=False)
        if len(missing_keys) != 0 or len(unexpected_keys) != 0:
            print("missing_keys: ", missing_keys)
            print("unexpected_keys: ", unexpected_keys)

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    return model

In [ ]:

def main(args):
    device = torch.device(args.device if torch.cuda.is_available() else "cpu")
    print("Using {} device training.".format(device.type))

    results_file = "/content/drive/MyDrive/TrainingStage2/save_weights/results{}.txt".format(datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

    data_transform = {
        "train": transforms.Compose([transforms.ToTensor(),
                                     transforms.RandomHorizontalFlip(0.5)]),
        "val": transforms.Compose([transforms.ToTensor()])
    }

    ###############################################---our dataset---###############################
    mydataset = args.data_path
    # check dataset root
    if os.path.exists(mydataset) is False:
        raise FileNotFoundError("MyDataSet dose not in path:'{}'.".format(mydataset))

    # load train data set
    # /content/dataset/train.txt
    train_dataset = MyDataSet(mydataset, "train", data_transform["train"], "train.txt")
    train_sampler = None
    ###############################################---End---###############################

    if args.aspect_ratio_group_factor >= 0:
        train_sampler = torch.utils.data.RandomSampler(train_dataset)
        group_ids = create_aspect_ratio_groups(train_dataset, k=args.aspect_ratio_group_factor)
        train_batch_sampler = GroupedBatchSampler(train_sampler, group_ids, args.batch_size)

    batch_size = args.batch_size
    nw = min([os.cpu_count(), batch_size if batch_size > 1 else 0, 8])  # number of workers
    print('Using %g dataloader workers' % nw)
    if train_sampler:
        train_data_loader = torch.utils.data.DataLoader(train_dataset,
                                                        batch_sampler=train_batch_sampler,
                                                        pin_memory=True,
                                                        num_workers=nw,
                                                        collate_fn=train_dataset.collate_fn)
    else:
        train_data_loader = torch.utils.data.DataLoader(train_dataset,
                                                        batch_size=batch_size,
                                                        shuffle=True,
                                                        pin_memory=True,
                                                        num_workers=nw,
                                                        collate_fn=train_dataset.collate_fn)

    ###############################################---our dataset-validation--###############################
    # load validation data set
    # /content/dataset/valid.txt
    val_dataset = MyDataSet(mydataset, "valid", data_transform["val"], "valid.txt")
    val_data_set_loader = torch.utils.data.DataLoader(val_dataset,
                                                      batch_size=1,
                                                      shuffle=False,
                                                      pin_memory=True,
                                                      num_workers=nw,
                                                      collate_fn=val_dataset.collate_fn)
    ###############################################---End--#################################################

    # create model num_classes equal background + 20 classes
    model = create_model(num_classes=args.num_classes + 1)
    # print(model)

    model.to(device)

    # define optimizer
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params,
                                lr=args.lr,
                                momentum=args.momentum,
                                weight_decay=args.weight_decay)

    scaler = torch.cuda.amp.GradScaler() if args.amp else None

    # learning rate scheduler
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                   step_size=3,
                                                   gamma=0.33)

    # resume
    if args.resume != "":
        checkpoint = torch.load(args.resume, map_location='cpu')
        model.load_state_dict(checkpoint['model'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        lr_scheduler.load_state_dict(checkpoint['lr_scheduler'])
        args.start_epoch = checkpoint['epoch'] + 1
        if args.amp and "scaler" in checkpoint:
            scaler.load_state_dict(checkpoint["scaler"])
        print("the training process from epoch{}...".format(args.start_epoch))

    train_loss = []
    learning_rate = []
    val_map_05 = []
    val_map_0595 = []

    for epoch in range(args.start_epoch, args.epochs):
        # train for one epoch, printing every 10 iterations
        mean_loss, lr = utils.train_one_epoch(model, optimizer, train_data_loader,
                                              device=device, epoch=epoch,
                                              print_freq=50, warmup=True,
                                              scaler=scaler)
        train_loss.append(mean_loss.item())
        learning_rate.append(lr)

        # update the learning rate
        lr_scheduler.step()

        # evaluate on the test dataset
        coco_info = utils.evaluate(model, val_data_set_loader, device=device)

        # write into txt
        with open(results_file, "a") as f:
            result_info = [f"{i:.4f}" for i in coco_info + [mean_loss.item()]] + [f"{lr:.6f}"]
            txt = "epoch:{} {}".format(epoch, '  '.join(result_info))
            f.write(txt + "\n")

        val_map_0595.append(coco_info[0])  # mAP @ IoU=0.50:0.95
        val_map_05.append(coco_info[1])  # mAP @ IoU=0.50

        # save weights
        save_files = {
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'lr_scheduler': lr_scheduler.state_dict(),
            'epoch': epoch}
        if args.amp:
            save_files["scaler"] = scaler.state_dict()
        torch.save(save_files, "/content/drive/MyDrive/TrainingStage2/save_weights/resNetFpn-model-{}.pth".format(epoch))

    # plot loss and lr curve
    if len(train_loss) != 0 and len(learning_rate) != 0:
        from plot_curve import plot_loss_and_lr
        plot_loss_and_lr(train_loss, learning_rate)

    # plot mAP curve
    if len(val_map_05) != 0 or len(val_map_0595) != 0:
        from plot_curve import plot_map
        plot_map(val_map_05, val_map_0595)

In [ ]:

if __name__ == "__main__":
    import argparse

    parser = argparse.ArgumentParser(
        description=__doc__)

    # device
    parser.add_argument('--device', default='cuda:0', help='device')
    parser.add_argument('--data-path', default='/content/dataset/', help='dataset')
    parser.add_argument('--num-classes', default=3, type=int, help='num_classes')
    parser.add_argument('--output-dir', default='/content/drive/MyDrive/TrainingStage2/save_weights', help='path where to save')
    parser.add_argument('--resume', default='', type=str, help='resume from checkpoint')
    parser.add_argument('--start_epoch', default=0, type=int, help='start epoch')
    parser.add_argument('--epochs', default=50, type=int, metavar='N', help='number of total epochs to run')
    parser.add_argument('--lr', default=0.02, type=float, help='initial learning rate, 0.02 is the default value for training on 8 gpus and 2 images_per_gpu')
    parser.add_argument('--momentum', default=0.9, type=float, metavar='M', help='momentum')
    parser.add_argument('--wd', '--weight-decay', default=1e-4, type=float, metavar='W', help='weight decay (default: 1e-4)', dest='weight_decay')
    parser.add_argument('--batch_size', default=4, type=int, metavar='N', help='batch size when training.')
    parser.add_argument('--aspect-ratio-group-factor', default=3, type=int)
    parser.add_argument("--amp", default=False, help="Use torch.cuda.amp for mixed precision training")

    opt = parser.parse_args(args=[])
    print(opt)

    if not os.path.exists(opt.output_dir):
        os.makedirs(opt.output_dir)

    main(opt)


Namespace(amp=False, aspect_ratio_group_factor=3, batch_size=4, data_path='/content/dataset/', device='cuda:0', epochs=50, lr=0.02, momentum=0.9, num_classes=3, output_dir='/content/drive/MyDrive/faster_rcnn/save_weights', resume='', start_epoch=0, weight_decay=0.0001)
Using cuda device training.
Using [0, 0.5, 0.6299605249474366, 0.7937005259840997, 1.0, 1.2599210498948732, 1.5874010519681994, 2.0, inf] as bins for aspect ratio quantization
Count of instances per bin: [  1   5 195   7  30 257 174  13]
Using 4 dataloader workers
_IncompatibleKeys(missing_keys=[], unexpected_keys=['fc.weight', 'fc.bias'])


/usr/local/lib/python3.7/dist-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Epoch: [0]  [  0/170]  eta: 0:22:49.960411  lr: 0.000138  loss: 2.2496 (2.2496)  loss_classifier: 1.0985 (1.0985)  loss_box_reg: 0.1021 (0.1021)  loss_objectness: 0.9964 (0.9964)  loss_rpn_box_reg: 0.0526 (0.0526)  time: 8.0586  data: 0.3313  max mem: 8398
Epoch: [0]  [ 50/170]  eta: 0:01:34.414789  lr: 0.006049  loss: 0.1942 (0.8097)  loss_classifier: 0.0547 (0.3658)  loss_box_reg: 0.1349 (0.2436)  loss_objectness: 0.0015 (0.1721)  loss_rpn_box_reg: 0.0032 (0.0282)  time: 0.6441  data: 0.0003  max mem: 9496
Epoch: [0]  [100/170]  eta: 0:00:49.940228  lr: 0.011961  loss: 0.6072 (0.6488)  loss_classifier: 0.1789 (0.2653)  loss_box_reg: 0.3694 (0.2524)  loss_objectness: 0.0396 (0.1075)  loss_rpn_box_reg: 0.0193 (0.0237)  time: 0.6345  data: 0.0003  max mem: 9496
Epoch: [0]  [150/170]  eta: 0:00:13.782865  lr: 0.017872  loss: 0.0878 (0.5845)  loss_classifier: 0.0285 (0.2260)  loss_box_reg: 0.0544 (0.2527)  loss_objectness: 0.0042 (0.0843)  loss_rpn_box_reg: 0.0008 (0.0215)  time: 0.6301  

KeyboardInterrupt: ignored